# Importing Functions 

In [1]:
import argparse
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
import time 
from torch.autograd import grad
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch import Tensor
device = torch.device('cuda')
seed = 0
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Models

In [2]:
class Encoder_ADDA(nn.Module):
    def __init__(self):
        super(Encoder_ADDA, self).__init__()
#         self.restored = False
        self.encoder = nn.Sequential(
            # first layer  4096*1-->  1017*8
            nn.Conv1d(1, 8, kernel_size=32,stride=2, padding=1),
#             nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.MaxPool1d(2),
            # second layer  1017*8-->  250*16
            nn.Conv1d(8, 16, kernel_size=16,stride=2, padding=1),
#             nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
            # third layer  250*16-->  60*32
            nn.Conv1d(16, 32, kernel_size=8,stride=2,padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2), 
            # fourth layer 60*32--> 14*32
            nn.Conv1d(32, 32, kernel_size=8,stride=2,padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            # fifth layer 14*32--> 3*64
            nn.Conv1d(32, 64, kernel_size=3,stride=2,padding=1),
#             nn.ReLU(),
            nn.MaxPool1d(2))
         # flatenning wit fully connected layers
        
        self.fc1 = nn.Linear(256, 256)# optimal when 0 source domain

    def forward(self, input):
        conv_out = self.encoder(input)
#         conv_out=F.dropout(conv_out)# we didn't need it when source domain is zero condition
        feat = self.fc1(conv_out.view(conv_out.shape[0],-1))
        return feat
            
    """classifier model for ADDA."""
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc2 = nn.Linear(256, 10) #this with 0 as souce it give optimal results 
  

    def forward(self, feat):
        out = F.dropout(F.relu(feat), training=self.training)
        out = self.fc2(out)
        return out
    
class Discriminator(nn.Module):
    """Discriminator model for source domain."""

    def __init__(self, input_dims, hidden_dims, output_dims):
        super(Discriminator, self).__init__()
#         self.restored = False
        self.layer = nn.Sequential(
            nn.Linear(input_dims, hidden_dims),
            nn.ReLU(),
            nn.Linear(hidden_dims, hidden_dims),
            nn.ReLU(),
            nn.Linear(hidden_dims, output_dims),
            nn.LogSoftmax()
        )

    def forward(self, input):
        
        out = self.layer(input)
        return out


src_ecoder=Encoder_ADDA().to(device)
tgt_encoder=Encoder_ADDA().to(device)
classifier= Classifier().to(device)

# Data Loading 

### Mixed Data 

In [3]:
# # Data Loading 
# wk_cond_a_full=torch.load('../data_5120L_new/wk_cond_a_full.pt')
# wk_cond_b_full=torch.load('../data_5120L_new/wk_cond_b_full.pt')
# wk_cond_c_full=torch.load('../data_5120L_new/wk_cond_c_full.pt')
# wk_cond_d_full=torch.load('../data_5120L_new/wk_cond_d_full.pt')

### Artificial Data

In [4]:
# artifcial=torch.load('../data_5120L_new/artificial_domains_raw')
# wk_cond_a_full,wk_cond_b_full,wk_cond_c_full,wk_cond_d_full=artifcial


### Real Data 

In [5]:
real=torch.load('../data_5120L_new/real_domains_raw.pt')
wk_cond_a_full,wk_cond_b_full,wk_cond_c_full,wk_cond_d_full=real

### Domain Shift Scenario

In [6]:
#selecting source and target domain 
source_domain,source_labels,test_data,test_labels=wk_cond_a_full
target_domain,target_labels,target_test,target_test_labels=wk_cond_b_full
# target_domain_1,target_labels_1,target_test_1,target_test_labels_1=wk_cond_c_full
# target_domain_2,target_labels_2,target_test_2,target_test_labels_2=wk_cond_d_full

sample_length=source_domain.size(1)
num_samples=source_domain.size(0)
num_test_samples=test_data.size(0)
num_target_samples= target_test.size(0)

# Parameters

In [7]:
"""Params for ADDA."""

# params for setting up models
model_root = "pytorch-adda"
d_input_dims = 256
d_hidden_dims = 256
d_output_dims = 2

# params for training network
num_epochs_pre = 30 # wK_a= 50; wk_B=100
log_step_pre = 5
eval_step_pre = 20
save_step_pre = 100
num_epochs = 5 # wk_a=20 
log_step = 1
save_step = 100
manual_seed = None

# params for optimizing models
d_learning_rate = 1e-4
c_learning_rate = 1e-4
c_init_learning_rate = 1e-4


beta1 = 0.5
beta2 = 0.9
bs=100 #20

# Pre-Training 

In [8]:


def train_src(encoder, classifier):
    """Train classifier for source domain."""
    ####################
    # 1. setup network #
    ####################
    # set train state for Dropout and BN layers
    src_encoder.train()
    classifier.train()

    # setup criterion and optimizer
    optimizer = optim.Adam(
        list(encoder.parameters()) + list(classifier.parameters()),
        lr=c_init_learning_rate,
        betas=(beta1, beta2))
    criterion = nn.CrossEntropyLoss()
    shuffled_indices=torch.randperm(num_samples)
    ####################
    # 2. train network #
    ####################
    t0 = time.time()

    for epoch in range(num_epochs_pre):
        running_loss=0
        running_accuracy=0
        num_batches=0
        shuffled_indices=torch.randperm(num_samples)

        for step in range(0,num_samples,bs):
            
            # shuffled data samples
            indices=shuffled_indices[step:step+bs]
            # training on target domain_a as a source
            minibatch_data =  Variable(source_domain[indices].unsqueeze(dim=1))
            minibatch_label=  Variable(source_labels[indices].squeeze())
            minibatch_data=minibatch_data.to(device)
            minibatch_label=minibatch_label.to(device)
#             inputs = (minibatch_data - src_mean)/src_std  
#             inputs=inputs.to(device)
    
            # zero gradients for optimizer
            optimizer.zero_grad()
            
            # compute loss for critic
            preds =classifier (encoder(minibatch_data.float()))
            loss = criterion(preds, minibatch_label)
            
            # optimize source classifier
            loss.backward()
            optimizer.step()
            running_loss += loss.detach().item()
        
            running_accuracy += (preds.max(1)[1] == minibatch_label).float().mean().item()
            num_batches+=1
        # print epoch info
        if ((epoch) % log_step_pre == 0):
            print("Epoch [{}/{}] : loss={} train_accuracy={}"
                  .format(epoch,
                          num_epochs_pre,
                          running_loss/num_batches,
                        running_accuracy*100/num_batches ))
       
        print('{} seconds'.format(time.time() - t0))
        # eval model on test set
        if ((epoch + 1) % eval_step_pre == 0):
            eval_src(encoder, classifier)
    return encoder, classifier


def eval_src(encoder, classifier):
    """Evaluate classifier for source domain."""
    # set eval state for Dropout and BN layers
    encoder.eval()
    classifier.eval()
    # init loss and accuracy
    loss = 0
    acc = 0
    num_batches=0
    mean_loss=0
    run_loss=0
    # set loss function
    criterion = nn.CrossEntropyLoss()
    x=[]
    y=[]
    # generate random vector of size 4000 to fit the gpu
#     shuffled_indices=torch.randperm(num_test_samples) #[0:4000]
    shuffled_indices=torch.randperm(num_test_samples) #[0:4000]

#     bs=20
    for i in range(0,num_test_samples,bs):
        indices=shuffled_indices[i:i+bs]
        minibatch_data =  test_data[indices].unsqueeze(dim=1)
        minibatch_label= test_labels[indices].squeeze()
#         print(minibatch_data.size())
#         print(minibatch_label.size())
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
#         inputs = (minibatch_data - src_mean)/src_std  
#         inputs=inputs.to(device)
        
        # forward pass
        scores=classifier(encoder(minibatch_data.float()))
        # calculate accuracy
                               
        acc += (scores.max(1)[1] == minibatch_label).float().mean().item()

        x.append(scores.max(1)[1])
        y.append(minibatch_label)
        loss = criterion(scores, minibatch_label)
        num_batches+=1
        run_loss += loss.detach().item()
#      
    mean_accuracy = acc / num_batches
#     acc_plt.append(mean_accuracy)
    mean_loss = run_loss / num_batches
    
    print("Avg Loss = {}, Avg Accuracy = {}".format( mean_loss, mean_accuracy*100))
    return (x,y)

### Adaptation Fucntion 

In [9]:
def train_tgt(src_encoder, tgt_encoder, critic):
    """Train encoder for target domain."""
    ####################
    # 1. setup network #
    ####################
    # set train state for Dropout and BN layers
    tgt_encoder.train()
    critic.train()
    # setup criterion and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer_tgt = optim.Adam(tgt_encoder.parameters(),
                               lr=c_learning_rate,
                               betas=(beta1, beta2))
    optimizer_critic = optim.Adam(critic.parameters(),
                                  lr=d_learning_rate,
                                  betas=(beta1, beta2))
    len_data_loader =num_samples
    t1 = time.time()

    ####################
    # 2. train network #
    ####################
    for epoch in range(num_epochs):
        # initilize loss
        run_critic_loss=0
        acc=0
        run_tgt_loss=0
        num_batches=0
        shuffled_indices=torch.randperm(num_samples)#[0:4000]
        # zip source and target data pair
        for i in range(0,num_samples,bs):
            ###########################
            # 2.1 train discriminator #
            ###########################
            indices=shuffled_indices[i:i+bs]
#             indices_T=shuffled_indices_T[i:i+bs]
            # make  variable
            sample_src= Variable(source_domain[indices].float().unsqueeze(dim=1)).to(device)
            sample_tgt_1=Variable(target_domain[indices].float().unsqueeze(dim=1)).to(device)
#             sample_tgt_2=Variable(target_domain_1[i:i+bs].float().unsqueeze(dim=1)).to(device)
#             sample_tgt_3=Variable(target_domain_2[i:i+bs].float().unsqueeze(dim=1)).to(device)

            # zero gradients for optimizer
            optimizer_critic.zero_grad()

            # extract and concat features
            feat_src = src_encoder(sample_src)
            feat_tgt_1 = tgt_encoder(sample_tgt_1)
#             feat_tgt_2 = tgt_encoder(sample_tgt_2)
#             feat_tgt_3 = tgt_encoder(sample_tgt_3)
#             feat_tgt= torch.cat((feat_tgt_1,feat_tgt_2),0)
#             feat_tgt= torch.cat((feat_tgt_1,feat_tgt_2,feat_tgt_3),0)
            feat_concat = torch.cat((feat_src, feat_tgt_1), 0)
    
            # predict on discriminator
            pred_concat = critic(feat_concat.detach()).to(device)
            # prepare real and fake lafeat_tgt_1bel
            label_src =Variable(torch.ones(feat_src.size(0)).long())
            label_tgt = Variable(torch.zeros(feat_tgt_1.size(0)).long())
            label_concat = torch.cat((label_src, label_tgt), 0).to(device)

            # compute loss for critic
#             print(pred_concat.size(), label_concat.size())
            loss_critic = criterion(pred_concat, label_concat)
            loss_critic.backward()

            # optimize critic
            optimizer_critic.step()

            pred_cls = torch.squeeze(pred_concat.max(1)[1])
            acc += (pred_cls == label_concat).float().mean()
            run_critic_loss+=  loss_critic.detach().item()
            
            ############################
            # 2.2 train target encoder #
            ############################
            optimizer_tgt.zero_grad() # edited here becareful 
            for i in range (1):
            # zero gradients for optimizer
#                 optimizer_critic.zero_grad()
                # extract and target features
                feat_tgt_1 = tgt_encoder(sample_tgt_1)
#                 feat_tgt_2 = tgt_encoder(sample_tgt_2)
#                 feat_tgt_3 = tgt_encoder(sample_tgt_3)
#                 feat_tgt= torch.cat((feat_tgt_1,feat_tgt_2),0)
#                 feat_tgt= torch.cat((feat_tgt_1,feat_tgt_2,feat_tgt_3),0)

                # predict on discriminator
                pred_tgt = critic(feat_tgt_1).to(device)

                # prepare fake labels to enforce the feature extractor to confuse the critic 
                label_tgt = Variable(torch.ones(feat_tgt_1.size(0)).long()).to(device)

                # compute loss for target encoder
                loss_tgt = criterion(pred_tgt, label_tgt)
                loss_tgt.backward()

                # optimize target encoder
                optimizer_tgt.step()

                run_tgt_loss+=loss_tgt.detach().item()
            num_batches+=1
            
        #######################
        # 2.3 print epoch info #
        #######################
        print('{} seconds'.format(time.time() - t1))
        if ((epoch) % log_step == 0):
            print("Epoch [{}/{}] :"
                  "discriminator_loss={:.5f} target_loss={:.5f} discriminator_acc={:.5f}"
                  .format(epoch,
                         num_epochs,
                          run_critic_loss/num_batches,
                          run_tgt_loss/(num_batches*5),
                          acc.data[0]/num_batches))
            print("=== Evaluating classifier for encoded target domain ===")
            print(">>> source only <<<")
            eval_tgt(src_encoder, src_classifier)
            print(">>> domain adaption <<<")
            eval_tgt(tgt_encoder, src_classifier)
    return tgt_encoder

def eval_tgt(encoder, classifier):
    """Evaluation for target encoder by source classifier on target dataset."""
    # set eval state for Dropout and BN layers
    encoder.eval()
    classifier.eval()
    # init loss and accuracy
    loss = 0
    acc = 0
    mean_loss=0
    mean_acc=0
    num_batches=0
    # set loss function
    criterion = nn.CrossEntropyLoss()
    x=[]
    y=[]
    shuffled_indices_T=torch.randperm(num_target_samples)
    # evaluate network
    with torch.no_grad():
        for i in range(0,num_target_samples,bs):
            indices_T=shuffled_indices_T[i:i+bs]
            minibatch_data =  target_test[indices_T].unsqueeze(dim=1)
            minibatch_label= target_test_labels[indices_T].squeeze()
            minibatch_data=minibatch_data.to(device)
            minibatch_label=minibatch_label.to(device)
            scores=classifier(encoder(minibatch_data))
            # calculate accuracy                     
            acc += (scores.max(1)[1] == minibatch_label).float().mean().item()
            loss += criterion(scores, minibatch_label)
            x.append(scores.max(1)[1])
            y.append(minibatch_label)
            num_batches+=1
    mean_accuracy = acc / num_batches
    mean_loss = loss / num_batches
    print("Avg Accuracy = {:2%}".format(mean_accuracy))
    return x,y

#  Main Code  

In [16]:
#selecting source and target domain 
source_domain,source_labels,test_data,test_labels=wk_cond_a_full
target_domain,target_labels,target_test,target_test_labels=wk_cond_b_full
# target_domain_1,target_labels_1,target_test_1,target_test_labels_1=wk_cond_b_full
# target_domain_2,target_labels_2,target_test_2,target_test_labels_2=wk_cond_d_full

sample_length=source_domain.size(1)
num_samples=source_domain.size(0)
num_test_samples=test_data.size(0)
num_target_samples= target_test.size(0)

num_epochs_pre = 50 # wK_a= 50; wk_B=100
num_epochs = 40 # wk_a=20

# # load models
src_encoder = Encoder_ADDA().to(device)

src_classifier= Classifier().to(device)

tgt_encoder = Encoder_ADDA().to(device)

critic = Discriminator(input_dims=d_input_dims,
                                  hidden_dims=d_hidden_dims,
                                  output_dims=d_output_dims).to(device)

# # train source model
src_encoder, src_classifier = train_src(
        src_encoder, src_classifier)
# src_encoder.load_state_dict(torch.load('src_enc_wk_d_temp.pt'))
# src_classifier.load_state_dict(torch.load('classifier_wk_d_temp.pt'))

tgt_encoder.load_state_dict(src_encoder.state_dict())
tgt_encoder = train_tgt(src_encoder, tgt_encoder, critic)

# eval target encoder on test set of target dataset
print("=== Evaluating classifier for encoded target domain ===")
print(">>> source only <<<")
eval_tgt(src_encoder, src_classifier)
print(">>> domain adaption <<<")
eval_tgt(tgt_encoder, src_classifier)

Epoch [0/50] : loss=1.6522717873255413 train_accuracy=46.24479043607911
1.221863031387329 seconds
2.352403402328491 seconds
3.4099910259246826 seconds
4.5279014110565186 seconds
5.600360631942749 seconds
Epoch [5/50] : loss=0.24748564961676797 train_accuracy=92.14322697371244
6.696540355682373 seconds
7.827008485794067 seconds
8.964200496673584 seconds
10.09201717376709 seconds
11.224376440048218 seconds
Epoch [10/50] : loss=0.0015600726648396328 train_accuracy=99.97916656235854
12.344244480133057 seconds
13.475347757339478 seconds
14.532756328582764 seconds
15.584980010986328 seconds
16.710729837417603 seconds
Epoch [15/50] : loss=0.0003616460010486359 train_accuracy=99.98958328117926
17.840832233428955 seconds
18.962018728256226 seconds
20.012866735458374 seconds
21.065526008605957 seconds
22.116299390792847 seconds
Avg Loss = 1.6511611602323882e-06, Avg Accuracy = 100.0
Epoch [20/50] : loss=0.0001382123486640315 train_accuracy=99.98958328117926
23.34103012084961 seconds
24.472038030

/home/mohamedragab1992/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:115: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Avg Accuracy = 92.366912%
4.190646409988403 seconds
Epoch [1/40] :discriminator_loss=0.69263 target_loss=0.14384 discriminator_acc=0.50541
=== Evaluating classifier for encoded target domain ===
>>> source only <<<
Avg Accuracy = 98.778364%
>>> domain adaption <<<
Avg Accuracy = 93.914102%
6.689856052398682 seconds
Epoch [2/40] :discriminator_loss=0.68389 target_loss=0.14647 discriminator_acc=0.53766
=== Evaluating classifier for encoded target domain ===
>>> source only <<<
Avg Accuracy = 98.774122%
>>> domain adaption <<<
Avg Accuracy = 94.763519%
9.073849201202393 seconds
Epoch [3/40] :discriminator_loss=0.68871 target_loss=0.14680 discriminator_acc=0.52241
=== Evaluating classifier for encoded target domain ===
>>> source only <<<
Avg Accuracy = 98.778364%
>>> domain adaption <<<
Avg Accuracy = 95.550369%
11.599729776382446 seconds
Epoch [4/40] :discriminator_loss=0.68936 target_loss=0.14799 discriminator_acc=0.51336
=== Evaluating classifier for encoded target domain ===
>>> sourc

Avg Accuracy = 99.975610%
77.60644602775574 seconds
Epoch [32/40] :discriminator_loss=0.69225 target_loss=0.14683 discriminator_acc=0.50553
=== Evaluating classifier for encoded target domain ===
>>> source only <<<
Avg Accuracy = 98.778364%
>>> domain adaption <<<
Avg Accuracy = 99.975610%
79.97050404548645 seconds
Epoch [33/40] :discriminator_loss=0.68930 target_loss=0.14771 discriminator_acc=0.51716
=== Evaluating classifier for encoded target domain ===
>>> source only <<<
Avg Accuracy = 98.778364%
>>> domain adaption <<<
Avg Accuracy = 100.000000%
82.42035293579102 seconds
Epoch [34/40] :discriminator_loss=0.69442 target_loss=0.14688 discriminator_acc=0.49129
=== Evaluating classifier for encoded target domain ===
>>> source only <<<
Avg Accuracy = 98.776243%
>>> domain adaption <<<
Avg Accuracy = 100.000000%
84.95316433906555 seconds
Epoch [35/40] :discriminator_loss=0.69180 target_loss=0.14724 discriminator_acc=0.51076
=== Evaluating classifier for encoded target domain ===
>>> 

([tensor([1, 0, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 0, 1, 2, 2, 2, 2, 1, 2, 2, 1,
          2, 1, 2, 2, 2, 2, 0, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 0, 0,
          1, 2, 2, 1, 2, 0, 1, 2, 1, 1, 1, 2, 1, 0, 1, 1, 0, 1, 2, 0, 2, 1, 1, 0,
          2, 1, 1, 1, 1, 2, 0, 0, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 0, 1, 1, 2, 2,
          0, 2, 2, 2], device='cuda:0'),
  tensor([0, 1, 2, 2, 1, 0, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 0, 0, 2, 2, 2, 2,
          2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1,
          2, 2, 2, 2, 2, 2, 1, 1, 0, 1, 2, 2, 0, 1, 2, 1, 1, 0, 0, 0, 2, 0, 1, 1,
          2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 0, 0, 1, 0, 1, 1, 2, 1, 1, 1,
          1, 2, 1, 2], device='cuda:0'),
  tensor([2, 2, 1, 1, 2, 2, 2, 1, 1, 0, 0, 1, 1, 1, 2, 2, 2, 0, 2, 1, 1, 2, 0, 2,
          1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 0, 1, 1, 2, 0, 1, 1, 1, 1, 2,
          2, 1, 2, 1, 1, 2, 0, 2, 2, 2, 0, 1, 1, 2, 0, 1, 0, 1, 0, 1, 2, 1, 1, 2,
          1, 1, 

In [11]:
# src_encoder.load_state_dict(torch.load('src_enc_wk_b_M_ADDA_new.pt'))
# src_classifier.load_state_dict(torch.load('classifier_wk_b_M_ADDA_mew.pt'))
# tgt_encoder.load_state_dict(torch.load('tgt_enc_wk_b_M_ADDA_new.pt'))
# # critic.load_state_dict(torch.load('critic_wk_b_M_ADDA_new.pt'))

target_domain,target_labels,target_test,target_test_labels=wk_cond_b_full

print(">>> domain adaption_B <<<")
print(">>> source only <<<")
x,y=eval_tgt(src_encoder, src_classifier)
print(">>> domain adaption <<<")
x,y=eval_tgt(tgt_encoder, src_classifier)

target_domain,target_labels,target_test,target_test_labels=wk_cond_c_full
print(">>> domain adaption _C<<<")
print(">>> source only <<<")
x,y=eval_tgt(src_encoder, src_classifier)
print(">>> domain adaption <<<")
x,y=eval_tgt(tgt_encoder, src_classifier)

target_domain,target_labels,target_test,target_test_labels=wk_cond_d_full
print(">>> domain adaption _D<<<")
print(">>> source only <<<")
x,y=eval_tgt(src_encoder, src_classifier)
print(">>> domain adaption <<<")
x,y=eval_tgt(tgt_encoder, src_classifier)


>>> domain adaption_B <<<
>>> source only <<<
Avg Accuracy = 99.973489%
>>> domain adaption <<<
Avg Accuracy = 100.000000%
>>> domain adaption _C<<<
>>> source only <<<
Avg Accuracy = 99.926829%
>>> domain adaption <<<
Avg Accuracy = 99.682926%
>>> domain adaption _D<<<
>>> source only <<<
Avg Accuracy = 100.000000%
>>> domain adaption <<<
Avg Accuracy = 100.000000%


In [12]:
# torch.save(tgt_encoder.state_dict(), 'tgt_enc_wk_d_artift.pt') 
# torch.save(critic.state_dict(), 'critic_wk_d_M_artif.pt') 
# torch.save(src_classifier.state_dict(), 'classifier_wk_d_atif.pt') 
# torch.save(src_encoder.state_dict(), 'src_enc_wk_d_artif.pt') 